In [ ]:
import pathlib
import numpy as np
import scipy as sp
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.patches
import mplstereonet
import skimage.exposure
import harmonica as hm
import verde as vd

import time

import sys
sys.path.append("..")
import micromag as mg #visit the git repository and clone it from there https://github.com/compgeolab/micromag-interfering-sources


Get the initial set up

In [ ]:
start = 0.01
stop = 30
num_points = 100  
heights = np.round(np.logspace(np.log10(start), np.log10(stop), num=num_points),2)
window_sizes = np.where(heights > 1.5, heights * 5, heights * 10)

Generate a list with all the spatial resolution used 

In [ ]:
spacings = (window_sizes * 2) / 100

In [ ]:
sensor_sample_distance = 5.0  # µm
coordinates = vd.grid_coordinates(
    region=[0, 2256-1.175, 0, 1410-1.175],  # µm
    spacing=1.175,  # µm
    extra_coords=sensor_sample_distance,
)

In [ ]:
data = vd.make_xarray_grid(coordinates, coordinates[2]*0, data_names=["bz"], dims=("y", "x"), extra_coords_names="z")
data.x.attrs = {"units": "µm"}
data.y.attrs = {"units": "µm"}
data.bz.attrs = {"long_name": "vertical magnetic field", "units": "nT"}


noise_std_dev = 50  # nT

data.bz.values += mg.qdm_noise(data.bz.shape, noise_std_dev, seed=42)


# Add a positive shift to the data (background in the shielded room)
data.bz.values += 400 # nT

data

In [ ]:
data.bz.plot.pcolormesh(cmap="seismic")#, vmax=10000, vmin=-10000)

In [ ]:
height_difference = 5

#upward continue 

data_up = (
    hm.upward_continuation(data.bz, height_difference)
    .assign_attrs(data.bz.attrs)
    .to_dataset(name="bz")
    .assign_coords(x=data.x, y=data.y)
    .assign_coords(z=data.z + height_difference)
)
data_up

In [ ]:
ax = data_up.bz.plot.pcolormesh(cmap="seismic",cbar_kwargs={
        "label": "bz (nT)"})#, vmax=50000, vmin=-50000)
plt.savefig('noise_50nT.pdf',dpi=96)

In [ ]:


# — your dirs —
data_dir   = pathlib.Path('/your/path')
fig_dir    = pathlib.Path('/your/path')
output_dir = pathlib.Path('/your/path')
output_dir.mkdir(exist_ok=True)

# — only seeds changed here — 
master_seed = 12345
rng         = np.random.default_rng(master_seed)
seeds       = rng.integers(0, 2**32, size=10, dtype=np.uint32)

for seed in seeds:
    # reproducible per-run RNG
    np.random.seed(int(seed))
    
    # isolate each run’s outputs so nothing ever gets re-added on top
    run_dir = output_dir / f'seed_{seed}'
    run_dir.mkdir(exist_ok=True)
    
    # — now exactly your original loop —
    for arquivo in sorted(data_dir.iterdir()):
        if arquivo.suffix.lower() != ".nc":
            continue
        name_base = arquivo.stem  
        new_name  = f"{name_base}_noise.nc"
        new_path  = run_dir / new_name        
        
        print(f"Run seed={seed!s}: Processing {arquivo.name}")
        data_source = xr.open_dataset(arquivo)

        # Generate a table for the noise grid
        grid_noise        = data['bz']
        grid_noise_table  = vd.grid_to_table(grid_noise)
        interpolator      = vd.KNeighbors()
        interpolator.fit(
            (grid_noise_table.x, grid_noise_table.y),
            grid_noise_table.bz
        )

        # Generate a table for the source data grid
        data_grid  = data_source['bz']
        shape      = np.shape(data_grid)
        data_table = vd.grid_to_table(data_grid)

        # Dimensions of the larger grid
        grid_height = data.x.values.max()
        grid_width  = data.y.values.max()

        # Determine window size
        x_min, x_max     = data_table['x'].min(), data_table['x'].max()
        y_min, y_max     = data_table['y'].min(), data_table['y'].max()
        window_width     = x_max - x_min
        window_height    = y_max - y_min

        # random shift, now reproducible
        x_start = np.random.uniform(0, grid_width  - window_width)
        y_start = np.random.uniform(0, grid_height - window_height)
        x_shift = x_start - x_min
        y_shift = y_start - y_min

        data_table['x_new'] = data_table['x'] + x_shift
        data_table['y_new'] = data_table['y'] + y_shift

        # Apply the interpolator and add noise
        data_table = data_table.assign(
            bz_new = interpolator.predict((data_table.x_new, data_table.y_new))
        )
        data_source.bz.values += np.reshape(data_table.bz_new, shape)

        # write out to this seed’s folder
        data_source.to_netcdf(new_path)
